In [19]:
import pymongo
from pymongo import MongoClient
client = MongoClient("localhost", 27017)
db = client["companies"]

from pandas.io.json import json_normalize
import json
import pandas as pd

data = db.nuevas_oficinas2.find()

In [20]:
dataset = pd.DataFrame(list(data))
dataset.head()

,_id,city,country_code,founded_year,geo,latitude,long-lat,longitude,name,total_money_raised,total_offices
0,5d26fafca7c6435d74091fab,Pleasanton,USA,2005,"{u'type': u'Point', u'coordinates': [-121.9049...",37.692934,"-121.904945, 37.692934",-121.904945,Zoho,$0,1
1,5d26fafca7c6435d74091fac,New York City,USA,2007,"{u'type': u'Point', u'coordinates': [-73.98550...",40.757929,"-73.985506, 40.757929",-73.985506,MeetMoi,$5.58M,1
2,5d26fafca7c6435d74091fad,London,GBR,2007,"{u'type': u'Point', u'coordinates': [-6.267494...",53.344104,"-6.267494, 53.344104",-6.267494,Babelgum,$13.2M,1
3,5d26fafca7c6435d74091fae,New York,USA,2006,"{u'type': u'Point', u'coordinates': [-74.00944...",40.746497,"-74.0094471, 40.7464969",-74.009447,Joost,$45M,1
4,5d26fafca7c6435d74091faf,San Francisco,USA,2004,"{u'type': u'Point', u'coordinates': [-122.3945...",37.764726,"-122.394523, 37.764726",-122.394523,Digg,$45M,1


In [21]:
def findNear(geopoint, radio_max_meters=1000):
    return db.nuevas_oficinas2.find({
        "geo": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })

def media_founded_year(lista_empresas): 
    result = 0
    for empresa in lista_empresas: 
        year = empresa.get('founded_year')
        #print(year, type(year))
        result += year
        
    return result/len(lista_empresas)


#media_founded_year(res)

In [22]:
def calcula(row):
    g=row.get('geo')
    lista=findNear(g)
    lista = list(lista)

    resultado=media_founded_year(lista)
    return resultado

calcula(dataset.loc[4])

2006

In [23]:
dataset['average_founded_year'] = dataset.apply(calcula, axis=1)

In [24]:
dataset

,_id,city,country_code,founded_year,geo,latitude,long-lat,longitude,name,total_money_raised,total_offices,average_founded_year
0,5d26fafca7c6435d74091fab,Pleasanton,USA,2005,"{u'type': u'Point', u'coordinates': [-121.9049...",37.692934,"-121.904945, 37.692934",-121.904945,Zoho,$0,1,2005
1,5d26fafca7c6435d74091fac,New York City,USA,2007,"{u'type': u'Point', u'coordinates': [-73.98550...",40.757929,"-73.985506, 40.757929",-73.985506,MeetMoi,$5.58M,1,2006
2,5d26fafca7c6435d74091fad,London,GBR,2007,"{u'type': u'Point', u'coordinates': [-6.267494...",53.344104,"-6.267494, 53.344104",-6.267494,Babelgum,$13.2M,1,2005
3,5d26fafca7c6435d74091fae,New York,USA,2006,"{u'type': u'Point', u'coordinates': [-74.00944...",40.746497,"-74.0094471, 40.7464969",-74.009447,Joost,$45M,1,2006
4,5d26fafca7c6435d74091faf,San Francisco,USA,2004,"{u'type': u'Point', u'coordinates': [-122.3945...",37.764726,"-122.394523, 37.764726",-122.394523,Digg,$45M,1,2006
5,5d26fafca7c6435d74091fb0,Sunnyvale,USA,2002,"{u'type': u'Point', u'coordinates': [-122.0551...",37.387845,"-122.055197, 37.387845",-122.055197,Plaxo,$28.3M,1,2003
6,5d26fafca7c6435d74091fb1,West Hollywood,USA,2006,"{u'type': u'Point', u'coordinates': [-118.3930...",34.090368,"-118.393064, 34.090368",-118.393064,Geni,$16.5M,1,2005
7,5d26fafca7c6435d74091fb2,New York,USA,2005,"{u'type': u'Point', u'coordinates': [-73.99643...",40.723731,"-73.9964312, 40.7237306",-73.996431,Wetpaint,$39.8M,2,2006
8,5d26fafca7c6435d74091fb3,San Francisco,USA,2006,"{u'type': u'Point', u'coordinates': [-122.4169...",37.776805,"-122.4169244, 37.7768052",-122.416924,Twitter,$1.16B,1,2006
9,5d26fafca7c6435d74091fb4,Menlo Park,USA,2004,"{u'type': u'Point', u'coordinates': [-122.1518...",37.416050,"-122.151801, 37.41605",-122.151801,Facebook,$2.43B,3,2005


In [25]:
dataset['average_founded_year'].max()

2013

In [26]:
#dataset = dataset[dataset['average_founded_year']==dataset["average_founded_year"].max()]

In [27]:
#df = df.drop(df[df["R1"] == 1].index)
dataset = dataset.drop(dataset[dataset["average_founded_year"] < 2013].index)

In [28]:
dataset

,_id,city,country_code,founded_year,geo,latitude,long-lat,longitude,name,total_money_raised,total_offices,average_founded_year
2324,5d26fafca7c6435d740928c1,Seattle,USA,2013,"{u'type': u'Point', u'coordinates': [-93.22650...",44.859587,"-93.226503, 44.859587",-93.226503,Gimigo,$0,1,2013
3993,5d26fafca7c6435d74092f47,Manchester,GBR,2013,"{u'type': u'Point', u'coordinates': [-2.243949...",53.483707,"-2.2439492, 53.4837067",-2.243949,Clowdy,$0,1,2013


In [14]:
dataset.to_json('afines4.json', orient="records", default_handler=str)

In [16]:
#composer require skagarwal/google-places-api

In [30]:
from googleplaces import GooglePlaces, types, lang

ImportError: No module named googleplaces

In [32]:
import requests
import os

API_KEY= ''
with open("token.py") as file:
    API_KEY = file.read().strip()
liststb=[]
for i in range(len(dataset)):
    solicitud = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius=1000&types=food&name={}&key={}'.format('longitude':-93.226503, 'latitude': 44.859587,Starbucks,API_KEY))
    stb= res.json()
    if len(stb)>0:
        liststb.append(i)

SyntaxError: invalid syntax (<ipython-input-32-aad887812636>, line 9)

In [12]:
#from dotenv import load_dotenv
#load_dotenv()
import requests
import os

#BASE_URL = "https://api.github.com"
BASE_URL = "https://maps.googleapis.com"
#https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key=YOUR_API_KEY
API_KEY = os.getenv("token_maps")

with open(".env") as file:
    API_KEY= file.read().strip()

name = "Starbucks"

res = requests.get("maps/api/place/findplacefromtext/json?input={}/%20of%20{}%20{}&inputtype")
res = requests.get("{}/maps/api/place/findplacefromtext/json?inputtype=textquery&fields={}&key={}".format(BASE_URL,name,API_KEY))
res = requests.get("{}/repos/{}/{}/stargazers".format(BASE_URL,owner, repo))


In [31]:
dataset.to_json('afines3.json', orient="records", default_handler=str)

In [98]:
#dataset = dataset[dataset['average_founded_year']==dataset["average_founded_year"].max()]


In [101]:
#dataset.to_csv("resultado_empresas", sep='\t', encoding='utf-8')

In [102]:
#dataset.to_json('nuevas_oficinas_geoquery.json', orient="records")

In [62]:

#dataset["average_founded_year"] = dataset["founded_year"].apply(media_founded_year)

In [ ]:
#dataset["founded_year"].value_counts()

In [ ]:
# iterate over rows with iterrows()
#for index, row in flights.head().iterrows():
     # access data using column names
 #    print(index, row['delay'], row['distance'], row['origin'])
    
#for index, row in dataset.head().iterrows():
 #   print(index, row)

In [ ]:
#dataset["average_founded_year"] = media_founded_year(dataset["founded_year"])
#def actualidad_score(col):
 #   actualidad_score = []
  #  for i in range(len(col)):
   #     actualidad_score.append(years_scores.get(col[i]))
    #return actualidad_score
#mydataframe["actualidad_score"] = actualidad_score(mydataframe["founded_year"])

In [ ]:
#media_founded_year(res)

In [ ]:
#dataset.head()

In [ ]:
# https://developers.google.com/maps/documentation/geocoding/intro

In [ ]:
#dataset.shape